# Case Análise PNAD

### Objetivo: 
#### Projetar modelo de aprendizagem de máquina para prever se uma pessoa possui acesso a internet via celular ou não.

#### Abaixo segue a tabela criada para a decisão de quais variáveis utilizar no modelo, para esta decisão foi levado em conta as condições:

Porcentagem de valores preenchidos maior que 70%.                                                                         
Desvios relevantes ( > 5 %).                                                                                              
Análise detalhada de dispersão dos dados de algumas colunas como uf e idade.                                               
Remoção de variáveis correlacionadas.                                                                                      

#### Tabela também presente no aquivo "Analise.xlsx".

*Para Melhor visualizar a tabela, selecione a célula.

Colunas PNAD          	                       | Valores não NaN. |  Desv. Médio	 |  Desv. Padrão  |	Utilizada no modelo
uf...............................................100,00%.............2,37%..............2,84%...........Sim
sexo.............................................100,00%.............1,47%..............2,07%...........Não
idade............................................100,00%.............0,55%..............0,61%...........Sim
condicao_domiciliar..............................100,00%.............12,89%.............15,23%..........Sim
raca.............................................100,00%.............18,84%.............22,26%..........Sim
estado_civil.................................... 59,39%..............14,25%.............19,99%..........Não
alfabetizado.....................................100,00%.............35,17%.............49,74%..........Sim
estuda...........................................100,00%.............22,68%.............32,07%..........Sim
internet_via_celular.............................100,00%.............5,23%..............7,39%...........Classificação
trabalha.........................................86,47%..............0,99%..............1,40%...........Não
idade_inicio_trabalho............................49,60%..............5,49%..............6,70%...........Não
responsavel_afazeres_domesticos..................86,47%..............18,18%.............25,71%..........Sim
horas_semanais_trabalho..........................43,30%..............6,08%..............8,38%...........Não
ativo_economicamente.............................86,47%..............10,82%.............15,31%..........Sim
ocupado..........................................50,39%..............25,19%.............35,63%..........Não
posicao_trabalho.................................50,18%..............4,49%..............6,07%...........Não
atividade........................................50,34%..............2,01%..............2,58%...........Não
ocupacao.........................................50,38%..............2,81%..............3,89%...........Não
tipo_familia.....................................100,00%.............1,72%..............1,49%...........Sim
situacao_censitaria..............................100,00%.............35,33%.............49,97%..........Sim
anos_de_estudo...................................100,00%.............6,57%..............8,51%...........Sim componentes_domicilio............................99,83%..............6,95%..............8,94%...........Sim rendimento_mensal................................96,38%..............9,13%..............11,10%..........Sim
nivel_de_instrucao...............................100,00%.............8,62%..............10,66%..........Sim
atividade_agricola...............................43,72%..............21,86%.............30,92%..........Não
inss.............................................100,00%.............36,58%.............51,73%..........Sim

#### Definir previsores e classes de acordo com as colunas selecionadas em análise prévia


In [1]:
import pandas as pd
previsores = pd.read_csv("pnad.csv", usecols={
        'alfabetizado',
        'anos_de_estudo',
        'ativo_economicamente',
        'componentes_domicilio',
        'condicao_domiciliar',
        'idade',
        'estuda',
        'inss',
        'nivel_de_instrucao',
        'raca',
        'rendimento_mensal',
        'responsavel_afazeres_domesticos',
        'situacao_censitaria',
        'tipo_familia',
        'uf'})

classes = pd.read_csv("pnad.csv", usecols={'internet_via_celular'}).values

#### Definir lista com colunas do tipo texto, e salvar seus valores distintos em uma lista "de-para".

In [2]:
colunas_texto = ['rendimento_mensal', 
                 'nivel_de_instrucao',
                 'raca',
                 'uf',
                 'condicao_domiciliar',
                 'tipo_familia',
                 'situacao_censitaria', 
                 'anos_de_estudo' ]

lista_de_para = []

for coluna in colunas_texto:
    tempListDistinct = list(set(previsores[coluna].values))
    lista_de_para.append({ coluna : tempListDistinct})

#### Realizar a alteração dos valores textos para cada indice da lista "de-para" para realizar o treinamento do modelo.

In [3]:
for coluna in colunas_texto:
    lista_valores_distintos = lista_de_para[colunas_texto.index(coluna)][coluna]
    for valor in lista_valores_distintos:
        previsores.loc[previsores[coluna] == valor, coluna] = lista_valores_distintos.index(valor)

#### Substituir valores NaN pela moda da coleção.

In [4]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy='most_frequent', axis=0)
imputer = imputer.fit(previsores)

previsores = imputer.transform(previsores)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Treinar modelo com 75% das amostras.

In [5]:
tamanho_treino = int(len(classes) * 0.75)

from sklearn.naive_bayes import GaussianNB
modelo = GaussianNB()
modelo.fit(previsores[:tamanho_treino], classes[:tamanho_treino])

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

#### Realizar previsões dos 25% restantes dos dados para validar modelo.

In [6]:
previsao = modelo.predict(previsores[tamanho_treino:])

#### Pegar resultados obtidos para validar porcentagem de acertos

In [7]:
dados_validacao = []
for resultado in classes[tamanho_treino:]: 
    dados_validacao.append(int(resultado[0]))

resultados = dados_validacao == previsao

print("Porcentagem de acertos: {}%".format(sum(resultados)* 100 / len (classes[tamanho_treino:])))

Porcentagem de acertos: 71.13285365252281%
